In [1]:
import requests

### Setting API parameters

In [ ]:
country_id = 13 # For the United States

# City keys
city1 = "Portland-South Portland-Biddeford"
city2 = "Portland-Vancouver-Beaverton"

# sensor id's
city1_ids = [855, 19840, 1195, 1923, 1083851]
city2_ids = [233585, 1067, 1068, 1069, 1502, 1527, 233545, 1064]

# Measurement parameters
measurement_param = "pm25"

# time period
start = "2024-10-06"
end = "2024-09-06"

### Running API Request

In [3]:
api_file = open("api_key.txt", "r")
key = api_file.read()
res = requests.get("https://api.openaq.org/v2/locations/2178", headers={"X-API-Key": key})

print(res)

### Testing City Pull

In [13]:
url = "https://api.openaq.org/v2/countries"
headers = {"accept": "application/json"}
city_list = requests.get(url, headers=headers)

print(city_list.text)

{"meta":{"name":"openaq-api","license":"","website":"/","page":1,"limit":100,"found":">100"},"results":[{"code":"AF","name":"Afghanistan","locations":2,"firstUpdated":"2019-10-20 22:30:00+00","lastUpdated":"2021-08-16 18:00:00+00","parameters":["pm25"],"count":23640,"cities":1,"sources":2},{"code":"DZ","name":"Algeria","locations":1,"firstUpdated":"2019-06-15 22:00:00+00","lastUpdated":"2024-06-11 01:00:00+00","parameters":["pm25"],"count":38951,"cities":1,"sources":1},{"code":"AQ","name":"Antarctica","locations":2,"firstUpdated":"2018-11-30 11:00:00+00","lastUpdated":"2021-02-04 14:00:00+00","parameters":["co","no2","o3","pm10","pm25","so2"],"count":9180,"cities":0,"sources":2},{"code":"AR","name":"Argentina","locations":17,"firstUpdated":"2016-01-01 03:00:00+00","lastUpdated":"2024-06-11 03:00:00+00","parameters":["co","no","no2","nox","o3","pm1","pm10","pm25","relativehumidity","so2","temperature","um003"],"count":294165,"cities":1,"sources":6},{"code":"AM","name":"Armenia","locatio

In [10]:
url = "https://api.openaq.org/v2/cities?limit=100&page=1&offset=0&sort=asc&order_by=city"
headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"meta":{"name":"openaq-api","license":"","website":"/","page":1,"limit":100,"found":4389},"results":[{"country":"DK","city":" ","count":23688,"locations":1,"firstUpdated":"2017-11-09 23:00:00+00","lastUpdated":"2024-04-05 08:00:00+00","parameters":["no2"]},{"country":"JP","city":" ","count":39192881,"locations":1537,"firstUpdated":"2023-07-14 17:00:00+00","lastUpdated":"2024-06-11 01:00:00+00","parameters":["co","no","no2","nox","pm25","so2"]},{"country":"KP","city":" ","count":33429,"locations":4,"firstUpdated":"2024-03-20 08:00:00+00","lastUpdated":"2024-06-11 01:00:00+00","parameters":["co","no2","o3","pm10","pm25","so2"]},{"country":"KR","city":" ","count":7049966,"locations":683,"firstUpdated":"2024-03-20 08:00:00+00","lastUpdated":"2024-06-11 01:00:00+00","parameters":["co","no2","o3","pm10","pm25","so2"]},{"country":"TT","city":" ","count":3925,"locations":1,"firstUpdated":"2024-03-20 01:00:00+00","lastUpdated":"2024-06-11 00:00:00+00","parameters":["co","pm10","pm25"]},{"count

### Returning for a City

In [9]:
url = "https://api.openaq.org/v2/averages?temporal=day&locations_id=70084&spatial=location&limit=100&page=1"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)

print(response.text)

{"meta":{"name":"openaq-api","license":"","website":"/","page":1,"limit":100,"found":0},"results":[]}


### Testing Data Pull

In [ ]:
url = "https://api.openaq.org/v2/averages?temporal=hour&locations_id=70084&spatial=location&limit=100&page=1"
headers = {"accept": "application/json"}
response = requests.get(url, headers=headers)
print(response.text)